### GWANW 2022 Bilby tutorial

This notebook walks you through configuring and running a GW parameter estimation job using the Bayesian inference package `Bilby`. This tutorial is based on this example, which looked at a different event: https://git.ligo.org/lscsoft/bilby/blob/master/examples/gw_examples/data_examples/GW150914.py

First we'll make sure we have installed the required Python packages

In [ ]:
import sys
!{sys.executable} -m pip install lalsuite gwosc gwpy bilby ipywidgets

import math
import bilby
from gwpy.timeseries import TimeSeries
from gwosc import datasets

For this tutorial we'll analyze one of the more notable events from the last observing run (O3), GW190521. This signal originated from a binary system of two black holes with masses of approximately 85 and 65 Msun, which merged to form a 142 Msun remnant black hole.

In [ ]:
# Get trigger time of the event
trigger_time = datasets.event_gps("GW190521")

# Analyze 4s of data around the trigger time
seglen = 4
post_trigger = 2
end_time = trigger_time + post_trigger
start_time = end_time - seglen

# Frequency band
flow = 11
fhigh = 512

# Options for PSD calculation
psd_seglen = 32 * seglen
psd_end_time = start_time
psd_start_time = start_time - psd_seglen
roll_off = 0.2
overlap = 0

# Detector list
detectors = ["H1", "L1", "V1"]

Next we fetch the strain data to be analyzed. We can do this using GWPy (see Adrian's tutorial)

In [ ]:
ifo_list = bilby.gw.detector.InterferometerList([])
for det in detectors:
    print(f"Fetching data for {det}")
    strain_data = TimeSeries.fetch_open_data(det, start_time, end_time)
    
    # Insert data into bilby interferometer class
    ifo = bilby.gw.detector.get_empty_interferometer(det)
    ifo.strain_data.set_from_gwpy_timeseries(strain_data)
    
    # Calculate the PSD. There is more than one way of doing this. The method shown
    # here involves looking at a long stretch of data before the analysis segment,
    # splitting it into several chunks, taking the spectrum of each and averaging
    # over all segments.
    psd_data = TimeSeries.fetch_open_data(det, psd_start_time, psd_end_time)
    psd_alpha = 2 * roll_off / seglen
    psd = psd_data.psd(
        fftlength=seglen, overlap=overlap, window=("tukey", psd_alpha), method="median")
    
    # Assign the PSD to the detector
    ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array=psd.frequencies.value, psd_array=psd.value)
    
    ifo.maximum_frequency = fhigh
    ifo.minimum_frequency = flow
    ifo_list.append(ifo)

Set the priors. `Bilby` includes several prior distributions that we can use, or we can also define our own.

For this example, we first load the default BBH priors, and then tailor the ranges to this particular event. In the interest of time, we will run a "minimal" example by fixing most of the parameters and sampling in just a few, but in actual analyses we usually allow all parameters to float.

In [ ]:
priors = bilby.gw.prior.BBHPriorDict()

# Sample in chirp mass and mass ratio with
# priors that give uniform distributions in the component masses
priors.pop("mass_1")
priors.pop("mass_2")
priors["chirp_mass"] = bilby.gw.prior.UniformInComponentsChirpMass(
    minimum=70, maximum=180, name="chirp_mass", latex_label=r"$\mathcal{M}$")
priors["mass_ratio"] = bilby.gw.prior.UniformInComponentsMassRatio(
    minimum=0.1, maximum=1, name="mass_ratio", latex_label=r"$q$")

# Luminosity distance prior is uniform in comoving source-frame volume
priors["luminosity_distance"] = bilby.gw.prior.UniformSourceFrame(
    minimum=1e2, maximum=1e5, name="luminosity_distance", latex_label=r"$d_L$", unit="Mpc")

# Set time prior to be +/- 0.1s around the trigger time
priors["geocent_time"] = bilby.core.prior.Uniform(
    minimum=trigger_time - 0.1, maximum=trigger_time + 0.1, name="geocent_time", latex_label=r"$t_{\rm c}$")

# Fix the remaining parameters to some appropriate 
priors["a_1"] = 0.4
priors["a_2"] = 0.42
priors["tilt_1"] = 0
priors["tilt_2"] = math.pi
priors["phi_12"] = 0
priors["phi_jl"] = 0
priors["psi"] = 3.13
priors["ra"] = 3.37
priors["dec"] = 0.46
priors["theta_jn"] = 0.93

Create the likelihood function for our analysis. The signal model is defined through the `waveform_generator` class, where we can set the approximant that will be used to reconstruct the signal. Then we pass this along with the priors defined above into a `GravitationalWaveTransient` likelihood.

For this specific case, the likelihood is already marginalized over time and distance, so ultimately we search over chirp mass and mass ratio.

In [ ]:
waveform_generator = bilby.gw.WaveformGenerator(
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments={"waveform_approximant": "IMRPhenomD",
                        "reference_frequency": 20, 
                        "minimum_frequency": flow})

likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list, waveform_generator, priors=priors,
    time_marginalization=True,
    phase_marginalization=True,
    distance_marginalization=True)

With everything set up, we can run the analysis. There is a helpful progress bar that you can monitor. In particular, take note of `dlogz`. This is an estimate of the total remaining evidence in the parameter space, which decreases (roughly) monotonically as the run progresses. The run terminates when `dlogz` drops below some threshold (default 0.1). The run will take about 2 hours to complete on your local machine. 

In [ ]:
outdir = "GW190521_minimal"
label = "GW190521_minimal"
bilby.core.utils.check_directory_exists_and_if_not_mkdir(outdir)

result = bilby.run_sampler(
    likelihood=likelihood, priors=priors,
    outdir=outdir, label=label,
    sampler="dynesty", nlive=1000, check_point_delta_t=600, 
    check_point_plot=True, npool=1,
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters,
    result_class=bilby.gw.result.CBCResult)

# Output plots
result.plot_corner(["chirp_mass", "mass_ratio"])
result.plot_waveform_posterior(interferometers=ifo_list)